<a href="https://colab.research.google.com/github/altiss/FNO/blob/main/iwe_m2d_win_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#VIC put here all the new imports that may be needed

from google.colab import drive

import numpy as np
import scipy.io
import h5py
import sklearn.metrics
import torch.nn as nn
from scipy.ndimage import gaussian_filter

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
# from utilities3 import *

import operator
from functools import reduce
from functools import partial

from timeit import default_timer

import os, os.path

import torch

v = torch.__version__
# assert(v[2] == '8')

In [ ]:

# mount my google drive to access dataset and save model
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab Notebooks"

Mounted at /content/drive
neural_solver


# Utils

In [ ]:
#VIC this is the content of: https://github.com/zongyi-li/fourier_neural_operator/blob/master/utilities3.py
# it may need to be udpated

#################################################
#
# Utilities
#
#################################################
# reading data
class MatReader(object):
    def __init__(self, file_path, to_torch=True, to_cuda=False, to_float=True):
        super(MatReader, self).__init__()

        self.to_torch = to_torch
        self.to_cuda = to_cuda
        self.to_float = to_float

        self.file_path = file_path

        self.data = None
        self.old_mat = None
        self._load_file()

    def _load_file(self):
        try:
            self.data = scipy.io.loadmat(self.file_path)
            self.old_mat = True
        except:
            self.data = h5py.File(self.file_path)
            self.old_mat = False

    def load_file(self, file_path):
        self.file_path = file_path
        self._load_file()

    def read_field(self, field):
        x = self.data[field]

        if not self.old_mat:
            x = x[()]
            x = np.transpose(x, axes=range(len(x.shape) - 1, -1, -1))

        if self.to_float:
            x = x.astype(np.float32)

        if self.to_torch:
            x = torch.from_numpy(x)

            if self.to_cuda:
                x = x.cuda()

        return x

    def set_cuda(self, to_cuda):
        self.to_cuda = to_cuda

    def set_torch(self, to_torch):
        self.to_torch = to_torch

    def set_float(self, to_float):
        self.to_float = to_float

# normalization, pointwise gaussian
class UnitGaussianNormalizer(object):
    def __init__(self, x, eps=0.00001):
        super(UnitGaussianNormalizer, self).__init__()

        # x could be in shape of ntrain*n or ntrain*T*n or ntrain*n*T
        self.mean = torch.mean(x, 0)
        self.std = torch.std(x, 0)
        self.eps = eps

    def encode(self, x):
        x = (x - self.mean) / (self.std + self.eps)
        return x

    def decode(self, x, sample_idx=None):
        if sample_idx is None:
            std = self.std + self.eps # n
            mean = self.mean
        else:
            if len(self.mean.shape) == len(sample_idx[0].shape):
                std = self.std[sample_idx] + self.eps  # batch*n
                mean = self.mean[sample_idx]
            if len(self.mean.shape) > len(sample_idx[0].shape):
                std = self.std[:,sample_idx]+ self.eps # T*batch*n
                mean = self.mean[:,sample_idx]

        # x is in shape of batch*n or T*batch*n
        x = (x * std) + mean
        return x

    def cuda(self):
        self.mean = self.mean.cuda()
        self.std = self.std.cuda()

    def cpu(self):
        self.mean = self.mean.cpu()
        self.std = self.std.cpu()

# normalization, Gaussian
class GaussianNormalizer(object):
    def __init__(self, x, eps=0.00001):
        super(GaussianNormalizer, self).__init__()

        self.mean = torch.mean(x)
        self.std = torch.std(x)
        self.eps = eps

    def encode(self, x):
        x = (x - self.mean) / (self.std + self.eps)
        return x

    def decode(self, x, sample_idx=None):
        x = (x * (self.std + self.eps)) + self.mean
        return x

    def cuda(self):
        self.mean = self.mean.cuda()
        self.std = self.std.cuda()

    def cpu(self):
        self.mean = self.mean.cpu()
        self.std = self.std.cpu()


# normalization, scaling by range
class RangeNormalizer(object):
    def __init__(self, x, low=0.0, high=1.0):
        super(RangeNormalizer, self).__init__()
        mymin = torch.min(x, 0)[0].view(-1)
        mymax = torch.max(x, 0)[0].view(-1)

        self.a = (high - low)/(mymax - mymin)
        self.b = -self.a*mymax + high

    def encode(self, x):
        s = x.size()
        x = x.view(s[0], -1)
        x = self.a*x + self.b
        x = x.view(s)
        return x

    def decode(self, x):
        s = x.size()
        x = x.view(s[0], -1)
        x = (x - self.b)/self.a
        x = x.view(s)
        return x

#loss function with rel/abs Lp loss
class LpLoss(object):
    def __init__(self, d=2, p=2, size_average=True, reduction=True):
        super(LpLoss, self).__init__()

        #Dimension and Lp-norm type are postive
        assert d > 0 and p > 0

        self.d = d
        self.p = p
        self.reduction = reduction
        self.size_average = size_average

    def abs(self, x, y):
        num_examples = x.size()[0]

        #Assume uniform mesh
        h = 1.0 / (x.size()[1] - 1.0)

        all_norms = (h**(self.d/self.p))*torch.norm(x.view(num_examples,-1) - y.view(num_examples,-1), self.p, 1)

        if self.reduction:
            if self.size_average:
                return torch.mean(all_norms)
            else:
                return torch.sum(all_norms)

        return all_norms

    def rel(self, x, y):
        num_examples = x.size()[0]

        diff_norms = torch.norm(x.reshape(num_examples,-1) - y.reshape(num_examples,-1), self.p, 1)
        y_norms = torch.norm(y.reshape(num_examples,-1), self.p, 1)

        if self.reduction:
            if self.size_average:
                return torch.mean(diff_norms/y_norms)
            else:
                return torch.sum(diff_norms/y_norms)

        return diff_norms/y_norms

    def __call__(self, x, y):
        return self.rel(x, y)

# A simple feedforward neural network
class DenseNet(torch.nn.Module):
    def __init__(self, layers, nonlinearity, out_nonlinearity=None, normalize=False):
        super(DenseNet, self).__init__()

        self.n_layers = len(layers) - 1

        assert self.n_layers >= 1

        self.layers = nn.ModuleList()

        for j in range(self.n_layers):
            self.layers.append(nn.Linear(layers[j], layers[j+1]))

            if j != self.n_layers - 1:
                if normalize:
                    self.layers.append(nn.BatchNorm1d(layers[j+1]))

                self.layers.append(nonlinearity())

        if out_nonlinearity is not None:
            self.layers.append(out_nonlinearity())

    def forward(self, x):
        for _, l in enumerate(self.layers):
            x = l(x)

        return x

# Model builder

In [ ]:
#VIC this is the content of: https://github.com/zongyi-li/fourier_neural_operator/blob/master/fourier_2d_time.py
# it needs to be udpated!
# i made a small modification to the original code, please try to preserve it when updating it
# the mod is highlighted by the following text #VIC-mod

torch.manual_seed(0)
np.random.seed(0)

################################################################
# fourier layer
################################################################

class SpectralConv2d_fast(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2):
        super(SpectralConv2d_fast, self).__init__()

        """
        2D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1 #Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.modes2 = modes2

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul2d(self, input, weights):
        # (batch, in_channel, x,y ), (in_channel, out_channel, x,y) -> (batch, out_channel, x,y)
        return torch.einsum("bixy,ioxy->boxy", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft2(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels,  x.size(-2), x.size(-1)//2 + 1, dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.modes1, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, :self.modes1, :self.modes2], self.weights1)
        out_ft[:, :, -self.modes1:, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, -self.modes1:, :self.modes2], self.weights2)

        #Return to physical space
        x = torch.fft.irfft2(out_ft, s=(x.size(-2), x.size(-1)))
        return x

class SimpleBlock2d(nn.Module):
    def __init__(self, modes1, modes2, width, t_in):
        super(SimpleBlock2d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .
        
        input: the solution of the previous 10 timesteps + 2 locations (u(t-10, x, y), ..., u(t-1, x, y),  x, y)
        input shape: (batchsize, x=64, y=64, c=12)
        output: the solution of the next timestep
        output shape: (batchsize, x=64, y=64, c=1)
        """

        self.modes1 = modes1
        self.modes2 = modes2
        self.width = width
        #self.fc0 = nn.Linear(12, self.width)
        # input channel is 12: the solution of the previous 10 timesteps + 2 locations (u(t-10, x, y), ..., u(t-1, x, y),  x, y)
        
        #VIC-mod t_in is passed as parameter now, so that we can decide the number of input time steps
        self.fc0 = nn.Linear(t_in+2, self.width)
        # input channel: the solution of the previous t_in timesteps + 2 locations (u(t-10, x, y), ..., u(t-1, x, y),  x, y)


        self.conv0 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv1 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv2 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv3 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.w0 = nn.Conv1d(self.width, self.width, 1)
        self.w1 = nn.Conv1d(self.width, self.width, 1)
        self.w2 = nn.Conv1d(self.width, self.width, 1)
        self.w3 = nn.Conv1d(self.width, self.width, 1)
        self.bn0 = torch.nn.BatchNorm2d(self.width)
        self.bn1 = torch.nn.BatchNorm2d(self.width)
        self.bn2 = torch.nn.BatchNorm2d(self.width)
        self.bn3 = torch.nn.BatchNorm2d(self.width)

        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        batchsize = x.shape[0]
        size_x, size_y = x.shape[1], x.shape[2]

        grid = self.get_grid(batchsize, size_x, size_y, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 3, 1, 2)

        x1 = self.conv0(x)
        x2 = self.w0(x.view(batchsize, self.width, -1)).view(batchsize, self.width, size_x, size_y)
        x = self.bn0(x1 + x2)
        x = F.relu(x)
        x1 = self.conv1(x)
        x2 = self.w1(x.view(batchsize, self.width, -1)).view(batchsize, self.width, size_x, size_y)
        x = self.bn1(x1 + x2)
        x = F.relu(x)
        x1 = self.conv2(x)
        x2 = self.w2(x.view(batchsize, self.width, -1)).view(batchsize, self.width, size_x, size_y)
        x = self.bn2(x1 + x2)
        x = F.relu(x)
        x1 = self.conv3(x)
        x2 = self.w3(x.view(batchsize, self.width, -1)).view(batchsize, self.width, size_x, size_y)
        x = self.bn3(x1 + x2)

        x = x.permute(0, 2, 3, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    def get_grid(self, batchsize, size_x, size_y, device):
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)

class Net2d(nn.Module):
    def __init__(self, modes, width, t_in):
        super(Net2d, self).__init__()

        """
        A wrapper function
        """

        self.conv1 = SimpleBlock2d(modes, modes, width, t_in)


    def forward(self, x):
        x = self.conv1(x)
        return x


    def count_params(self):
        c = 0
        for p in self.parameters():
            c += reduce(operator.mul, list(p.size()))

        return c

# Dataset Loader

In [ ]:
#VIC leave this as it is

def dataset_loader(dataset_name, dataset_path, n, win, stride=1, win_lim=-1) :
  # get N, T, w and h from file name
  datadetails = dataset_name.split("_")
  N = datadetails[2][1:]
  N = int(N) # num of dataset entries
  T = datadetails[3][1:]
  T = int(T) # timesteps of each dataset entry
  w = datadetails[4][1:]
  w = int(w)
  h = int(w)
  # all the other parameters are dataset specific!

  # to grab only a subset of the timesteps in each data entry
  if win_lim != -1 and win_lim < T:
    T = win_lim

  # check that window size is smaller than number of timesteps per each data entry
  assert (T >= win)  

  # each entry in the dataset is now split in several trainig points, as big as T_in+T_out
  #p_num = T-(win-1) # number of points per each dataset entry  
  p_num = int( (T-win)/stride ) +1 # number of points per each dataset entry  
  p_tot = N * p_num # all training points in dataset
  print('Availble points in dataset: ', p_tot)
  print('Points requested: ', n)
  assert (p_tot >= n)  

  # count number of checkpoints, their size and check for remainder file
  dataset_full_path = dataset_path + dataset_name + '/'

  files = os.listdir(dataset_full_path)
  cp = len(files)
  rem = 0

  for name in files :
    splitname = name.split("_")
    if splitname[-2] == 'rem' :
      rem = 1
      cp = cp-1
      break

  files = sorted(files) # order checkpoint files
  cp_size = files[0].split("_")[-1].split(".")[0] # read number of dataset entries in each checkpoint
  cp_size = int(cp_size)
  #cp_size = N//cp # number of dataset entries in each checkpoint
  rem_size = 0 # number of dataset entries in remainder file [if any]
  if rem > 0 :
    rem_size = N - (cp*cp_size)

  #print(N, T, w, h, cp, cp_size, rem, rem_size)

  # prepare tensor where to load requested data points
  u = torch.zeros(n, h, w, win)
  #print(u.shape)

  # actual sizes with moving window

  cp_size_p = cp_size * p_num # number of points per each check point
  rem_size_p = rem_size * p_num # number of points in remainder

  # let's load

  # check how many files we need to cover n points
  full_files = n//(cp_size_p)
  extra_datapoints = n%cp_size_p

  extra_file_needed = extra_datapoints>0

  print('Retrieved over', full_files, 'full files,', cp_size_p, 'points each')

  # check that all numbers are fine
  assert (full_files+extra_file_needed <= cp+rem)

  
  #print(files)
  

  # first load from files we will read completely 
  cnt = 0
  for f in range(0,full_files) :
    dataloader = MatReader(dataset_full_path+files[f])
    uu = dataloader.read_field('u')
    #print(f, files[f])
    # unroll all entries with moving window
    for e in range(0, cp_size) :
      # window extracts p_num points from each dataset entry
      #print('e', e, 'p_num', p_num)
      for tt in range(0, p_num) :
        #print('tt', tt)
        t = tt*stride
        #print('t', t)
        u[cnt:cnt+1,...] = uu[e,:,:,t:t+win]
        cnt = cnt+1

  #print(cnt, extra_datapoints)
  

  # then load any possible remainder from a further file
  if extra_datapoints>0 :
    print('Plus', extra_datapoints, 'points from further file')
    extra_entries = (extra_datapoints+0.5)//p_num # ceiling to be sure to have enough entries to unroll
    dataloader = MatReader(dataset_full_path+files[full_files])
    uu = dataloader.read_field('u')
    entry = -1
    while cnt < n :
      entry = entry+1
      for tt in range(0,p_num) :
        t = tt*stride
        u[cnt:cnt+1,...] = uu[entry,:,:,t:t+win] 
        cnt = cnt+1
        if cnt >= n :
          break

  return u

# Settings

In [ ]:
#VIC leave this as it is
# then you will be able to play around with the settings

#-------------------------------------------------------------------------------
# editable simulation parameters
ntrain = 15000
ntest = 2000

modes = 12
width = 32

batch_size = 20

epochs = 500
learning_rate = 0.0025
scheduler_step = 100
scheduler_gamma = 0.5

print(epochs, learning_rate, scheduler_step, scheduler_gamma)

T_in = 10
T_out = 10
# T_in+T_out is window size!

win_stride = 1
win_lim = -1 #(T_in+T_out)*200 #-1 for no limit

# dataset
dataset_name = 'iwe_d1_n1000_t50_s64_mu0@1_rho0@5_gamma1_xm2_xM32'

#-------------------------------------------------------------------------------



MODEL_ID = '2d_win'

dataset_path = '/content/drive/My Drive/Colab Notebooks/neural_solver/wave_equation/irreducible/datasets/'

# retrieve dataset details and check them
splitname = dataset_name.split('_')

DATASET = splitname[1]

S = splitname[4]
S = int(S[1:])

mu = splitname[5][2:]
rho = splitname[6][3:]
gamma = splitname[7][5:]

# prepare to save model
model_name = 'iwe_m'+MODEL_ID+'_'+DATASET+'_n'+str(ntrain)+'+'+str(ntest)+'_e'+str(epochs)+'_m'+str(modes)+'_w'+ str(width)+'_ti'+str(T_in)+'_to'+str(T_out)+'_ws'+str(win_stride)+'_wl'+str(win_lim)+'_s'+str(S)+'_m'+mu+'_r'+rho+'_g'+gamma

model_path = dataset_path[:-9]+'/models/'

500 0.0025 100 0.5


# Load Data

In [ ]:
#VIC i think this needs only a minor update, i.e., removing the padding of the location

t1 = default_timer()

u = dataset_loader(dataset_name, dataset_path, ntrain+ntest, T_in+T_out, win_stride, win_lim)

train_a = u[:ntrain,:,:,:T_in]
train_u = u[:ntrain,:,:,T_in:T_in+T_out]

ntest_start = ntrain
test_a = u[ntest_start:ntest_start+ntest,:,:,:T_in]
test_u = u[ntest_start:ntest_start+ntest:,:,:,T_in:T_in+T_out]

#test_a = u[-ntest:,:,:,:T_in]
#test_u = u[-ntest:,:,:,T_in:T_in+T_out]


print(train_u.shape, test_u.shape)
assert (S == train_u.shape[-2])
assert (T_out == train_u.shape[-1])

train_a = train_a.reshape(ntrain,S,S,T_in)
test_a = test_a.reshape(ntest,S,S,T_in)

#VIC should be removed
# pad the location (x,y)
# gridx = torch.tensor(np.linspace(0, 1, S), dtype=torch.float)
# gridx = gridx.reshape(1, S, 1, 1).repeat([1, 1, S, 1])
# gridy = torch.tensor(np.linspace(0, 1, S), dtype=torch.float)
# gridy = gridy.reshape(1, 1, S, 1).repeat([1, S, 1, 1])

# train_a = torch.cat((gridx.repeat([ntrain,1,1,1]), gridy.repeat([ntrain,1,1,1]), train_a), dim=-1)
# test_a = torch.cat((gridx.repeat([ntest,1,1,1]), gridy.repeat([ntest,1,1,1]), test_a), dim=-1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_a, train_u), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=False)

t2 = default_timer()

print('preprocessing finished, time used:', t2-t1, 's')
print('train input shape:',train_a.shape, ' output shape: ', train_u.shape)    

Availble points in dataset:  31000
Points requested:  17000
Retrieved over 2 full files, 6200 points each
Plus 4600 points from further file
torch.Size([15000, 64, 64, 10]) torch.Size([2000, 64, 64, 10])
preprocessing finished, time used: 27.508791729999984 s
train input shape: torch.Size([15000, 64, 64, 10])  output shape:  torch.Size([15000, 64, 64, 10])


# Build and Train

In [ ]:
#VIC this needs a couple of touch ups, as at the bottom of: https://github.com/zongyi-li/fourier_neural_operator/blob/master/fourier_2d_time.py
# also, i made very minor changes to the original code, to make the logic clearer

if torch.cuda.is_available() :
  model = Net2d(modes, width, T_in).cuda()
  device  = torch.device('cuda')
else :
  model = Net2d(modes, width, T_in)
  device  = torch.device('cpu')


print(model.count_params())
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)


# myloss = LpLoss(size_average=False)

#VIC these are not needed anymore
# gridx = gridx.to(device)
# gridy = gridy.to(device)



myloss = LpLoss(size_average=False)
step = 1
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2_step = 0
    train_l2_full = 0
    for xx, yy in train_loader:
        loss = 0
        xx = xx.to(device)
        yy = yy.to(device)

        for t in range(0, T_out, step):
            y = yy[..., t:t + step]
            im = model(xx)
            loss += myloss(im.reshape(batch_size, -1), y.reshape(batch_size, -1))

            if t == 0:
                pred = im
            else:
                pred = torch.cat((pred, im), -1)

            xx = torch.cat((xx[..., step:], im), dim=-1)

        train_l2_step += loss.item()
        l2_full = myloss(pred.reshape(batch_size, -1), yy.reshape(batch_size, -1))
        train_l2_full += l2_full.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    test_l2_step = 0
    test_l2_full = 0
    with torch.no_grad():
        for xx, yy in test_loader:
            loss = 0
            xx = xx.to(device)
            yy = yy.to(device)

            for t in range(0, T_out, step):
                y = yy[..., t:t + step]
                im = model(xx)
                loss += myloss(im.reshape(batch_size, -1), y.reshape(batch_size, -1))

                if t == 0:
                    pred = im
                else:
                    pred = torch.cat((pred, im), -1)

                xx = torch.cat((xx[..., step:], im), dim=-1)

            test_l2_step += loss.item()
            test_l2_full += myloss(pred.reshape(batch_size, -1), yy.reshape(batch_size, -1)).item()

    t2 = default_timer()
    scheduler.step()
    print(ep, t2 - t1, train_l2_step / ntrain / (T_out / step), train_l2_full / ntrain, test_l2_step / ntest / (T_out / step),
          test_l2_full / ntest)
    
# add loss to name, with 4 decimals    
final_training_loss = '{:.4f}'.format(test_l2_full / ntest)
final_training_loss = final_training_loss.replace('.', '@')

model_name = model_name+'_loss'+final_training_loss   
model_full_path = model_path+model_name

torch.save(model, model_full_path)

#path_train_err = model_path+'results/'+model_name+'_train.txt'
#path_test_err = model_path+'results/'+model_name+'_test.txt'

1188897
0 197.23994342700001 1.4070204296875 1.4006416889190674 1.5094630561828613 1.5289709177017212
1 196.75714442900005 0.7846910990397136 0.7844967452367146 0.8060891204833984 0.7991349730491638
2 196.73732774400003 0.6192303081766763 0.6138920731226604 0.6789006214141846 0.6710995998382568
3 196.619039102 0.5462406439208984 0.5417594748814901 0.618939658355713 0.6131355381011963
4 196.510104179 0.5190478286234538 0.5136619705200195 0.6197721771240234 0.6123944118022918
5 196.57225777999997 0.4968086539713542 0.4914577607790629 0.5611893363952637 0.5563551433086396
6 196.59250822499985 0.47166127182006834 0.46585251592000326 0.5330247856140137 0.5285565721988678
7 196.59861907599998 0.4577510762023926 0.4510818262418111 0.5234816263198853 0.5169168884754181
8 196.60028257399995 0.4362749289703369 0.4286818249384562 0.5187018566131592 0.5133547606468201
9 196.55137704799995 0.4118010817718506 0.4038964545249939 0.48086801280975344 0.4742453033924103
10 196.65946733400006 0.401348237

To avoid automatic runtime disconnection for inactivity [90 minutes]:

_press ctlr+shift+i

_then go to the console and type:

```
function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
setInterval(ClickConnect, 60000)
```


This js code will click the "Connect" button [top right of Colab notebook] every minute...
Maximum runtime life remains 12 hours

Found here: https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting?page=1&tab=votes#tab-top